<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/03_xyz_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-25 08:37:55--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12099 (12K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.82K  --.-KB/s    in 0s      

2022-06-25 08:37:55 (90.7 MB/s) - ‘utils.py’ saved [12099/12099]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [1]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import ModelCreator, get_train_val_ds, get_class_weight

In [2]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b3/feature_vector/2'

In [3]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [4]:
train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 17043 files belonging to 16 classes.
Found 3495 files belonging to 16 classes.


In [5]:
classes, class_weight = get_class_weight()

In [6]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tfio.experimental.color.rgb_to_xyz(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tfio.experimental.color.rgb_to_xyz(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [7]:
xyz_creator = ModelCreator(HUB_URL, 'XYZ-model')
xyz_model = xyz_creator.make_model(img_size=IMG_SIZE)

Model: "XYZ-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1536)              12930622  
                                                                 
 dense (Dense)               (None, 16)                24592     
                                                                 
Total params: 12,955,214
Trainable params: 12,845,998
Non-trainable params: 109,216
_________________________________________________________________


In [8]:
callbacks = xyz_creator.get_callbacks()

In [9]:
history = xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=15,
                        class_weight=class_weight,
                        validation_data=val_ds)

Epoch 1/15
533/533 [==============================] - 1706s 3s/step - loss: 17.8534 - auc: 0.7334 - accuracy: 0.6543 - val_loss: 1.1138 - val_auc: 0.7104 - val_accuracy: 0.5957
Epoch 2/15
533/533 [==============================] - 259s 483ms/step - loss: 10.3508 - auc: 0.8794 - accuracy: 0.7938 - val_loss: 0.7962 - val_auc: 0.8333 - val_accuracy: 0.7597
Epoch 3/15
533/533 [==============================] - 258s 482ms/step - loss: 7.9526 - auc: 0.9151 - accuracy: 0.8361 - val_loss: 0.7981 - val_auc: 0.8421 - val_accuracy: 0.7668
Epoch 4/15
533/533 [==============================] - 266s 496ms/step - loss: 6.0219 - auc: 0.9402 - accuracy: 0.8721 - val_loss: 1.0013 - val_auc: 0.7940 - val_accuracy: 0.7439
Epoch 5/15
533/533 [==============================] - 257s 480ms/step - loss: 4.5287 - auc: 0.9605 - accuracy: 0.9010 - val_loss: 0.8910 - val_auc: 0.8208 - val_accuracy: 0.7459
Epoch 6/15
533/533 [==============================] - 256s 477ms/step - loss: 3.2181 - auc: 0.9738 - accuracy:

In [10]:
!cp ./logs/checkpoints/XYZ-model.h5 ./drive/MyDrive/checkpoints/xyz-model-final.h5